In [10]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import gensim
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import numpy as np
from gensim.models import CoherenceModel

DATA_PATH = "../data"
VIS_PATH = "../visualisations"
# model = spacy.load("en_core_web_trf")

In [11]:
data = pd.read_pickle(DATA_PATH + "/processed/dataset_tokenized.pkl")
data.head()

,Contract,Title,Type of contract,Short description,Total value,Place of performance,Contractor,Short description tokenized
0,000007-2017,Framework contract for the provision of passen...,SERVICES,Provision of passenger transfer services by bu...,150000.0,GR,[Vagia Tzislaki — Lesvorama General Travel Age...,provision passenger transfer service bus greec...
1,000007-2017,Framework contract for the provision of passen...,SERVICES,Provision of passenger services by bus in Gree...,150000.0,GR,[Vagia Tzislaki — Lesvorama General Travel Age...,provision passenger service bus greece chios i...
2,000007-2017,Framework contract for the provision of passen...,SERVICES,Framework contract for the provision of passen...,NaN,GR,None,framework contract provision passenger transfe...
3,008449-2018,Provision of HID® FARGO® HDP5000 ID card print...,SUPPLIES,The subject of the contract is provision of HI...,190000.0,PL,[Control system FMN Sp z oo],subject contract provision hid ® fargo ® hdp50...
4,008450-2018,Provision of painting services for Frontex,SERVICES,Provision of painting services for Frontex.,NaN,PL,None,provision painting service frontex


In [12]:
tokenized_descriptions = data[data["Short description tokenized"].notna()]["Short description tokenized"].tolist()

docs = [desc.split(" ") for desc in tokenized_descriptions]
id2word = gensim.corpora.Dictionary(docs)
corpus = [id2word.doc2bow(doc) for doc in docs]

In [16]:
n_topics = 11
lda_model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=n_topics,
    decay=0.7,
    offset=10,
    chunksize=128,
    iterations=10,
    alpha=np.array([1/n_topics for i in range(n_topics)]),
    eta=1/n_topics
)

c = CoherenceModel(lda_model, texts=data[data["Short description tokenized"].notna()]["Short description tokenized"].str.split(" ").values)

c.get_coherence()

BrokenPipeError: [Errno 32] Broken pipe

In [9]:
pprint(lda_model.print_topics())
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='tsne')

[(0,
  '0.013*"software" + 0.011*"service" + 0.008*"frontex" + 0.007*"shall" + '
  '0.007*"development" + 0.007*"system" + 0.005*"contract" + '
  '0.005*"surveillance" + 0.005*"external" + 0.005*"technical"'),
 (1,
  '0.009*"service" + 0.006*"associate" + 0.005*"container" + 0.005*"toilet" + '
  '0.005*"frontex" + 0.004*"renting" + 0.003*"contract" + 0.003*"provision" + '
  '0.003*"framework" + 0.002*"system"'),
 (2,
  '0.006*"service" + 0.004*"/" + 0.004*"system" + 0.003*"support" + '
  '0.003*"contract" + 0.003*"frontex" + 0.003*"maintenance" + 0.002*"unisex" + '
  '0.002*"shall" + 0.002*"lot"'),
 (3,
  '0.006*"software" + 0.006*"contract" + 0.006*"support" + 0.005*"service" + '
  '0.004*"frontex" + 0.004*"system" + 0.003*"maintenance" + 0.003*"situation" '
  '+ 0.003*"framework" + 0.002*"development"'),
 (4,
  '0.010*"course" + 0.006*"polish" + 0.006*"level" + 0.005*"contract" + '
  '0.005*"frontex" + 0.005*"service" + 0.004*"b2" + 0.003*"survival" + '
  '0.003*"language" + 0.003*"p

In [16]:
vis

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      -48.478802  173.279541       1        1  30.016962
8     -177.662354   71.692581       2        1  28.618053
10     -47.803089 -198.148544       3        1  21.063652
9       83.528748  -85.487503       4        1   7.031708
4      224.009064   52.914520       5        1   3.289326
1     -188.916626 -108.940475       6        1   2.689069
6      239.077667 -110.762962       7        1   2.664857
5      113.183067 -240.686661       8        1   1.861713
7       63.760567   59.448547       9        1   1.128862
2      -54.386539  -26.327744      10        1   0.989986
0      127.195404  201.366089      11        1   0.645814, topic_info=             Term        Freq       Total Category  logprob  loglift
31        frontex  107.000000  107.000000  Default  30.0000  30.0000
6         service  180.000000  180.000000  Default  29.0000  29.0000
66   surveillance   46.000000   46.000000  Default  28.0000  28.0000
249      software  107.000000  107.000000  Default  27.0000  27.0000
145        system   48.000000   48.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
216       include    0.079344   16.927218  Topic11  -6.6697  -0.3205
28         tender    0.081481   30.278765  Topic11  -6.6431  -0.8754
249      software    0.084367  107.601874  Topic11  -6.6083  -2.1086
11      framework    0.081094   62.158274  Topic11  -6.6479  -1.5994
14            lot    0.077651   31.574956  Topic11  -6.6912  -0.9655

[708 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
168       1  0.503732          -
168       2  0.209888          -
168       3  0.125933          -
168       4  0.083955          -
168       5  0.041978          -
...     ...       ...        ...
284       2  0.402520  workplace
284       3  0.201260  workplace
906       1  0.229795     wp.2.2
906       2  0.229795     wp.2.2
906       3  0.229795     wp.2.2

[1527 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 11, 10, 5, 2, 7, 6, 8, 3, 1])